In [1]:
import time
import pandas as pd
start_time = time.time()
# Sets
students = pd.read_csv('set_students.csv')['students'].tolist()
teachers = pd.read_csv('set_teachers.csv')['teachers'].tolist()
courses = pd.read_csv('set_courses.csv')['courses'].tolist()
timeslots = pd.read_csv('set_timeslots.csv')['timeslots'].tolist()
living_units = pd.read_csv('set_living_units.csv')['living Units'].tolist()

# Parameters
H = pd.read_csv('student_total_hours_without_varia.csv', sep=';').set_index('students').to_dict()['H_s']
m = pd.read_csv('student_courses_min_hours_without_varia.csv', sep=';').set_index(['students', 'courses']).to_dict()['min hours']
M = pd.read_csv('student_courses_max_hours_without_varia.csv', sep=';').set_index(['students', 'courses']).to_dict()['max hours']
V = pd.read_csv('student_courses_without_varia.csv', sep = ';').set_index('students').groupby('students')['courses'].apply(list).to_dict()
R = (pd.read_csv('student_teacher_relation.csv', sep=';')
    .pivot(index='Student', columns='teacher', values='relation')
    .fillna(0)
    .astype(int)
    .to_dict(orient='index')
)
T_avail_u = pd.read_csv('living_unit_hours.csv', sep=';').set_index('living units').groupby('living units')['timeslots'].apply(set).to_dict()
u_s = pd.read_csv('student_living_unit.csv', sep=';').set_index('students').groupby('students')['living units'].apply(set).to_dict()
T_unavail_s = pd.read_csv('student_unavailable.csv', sep=';').set_index('students').groupby('students')['unavailable'].apply(set).to_dict()
T_avail = {}

for student, living_unit_set in u_s.items():
    # Get the single living unit from the set
    living_unit = next(iter(living_unit_set), None)
    
    if not living_unit:
        T_avail[student] = set()
        continue

    # Timeslots available for the living unit
    lu_timeslots = T_avail_u.get(living_unit, set())

    # Timeslots the student is unavailable
    unavailable = T_unavail_s.get(student, set())

    # Subtract unavailable from living unit's available
    available = lu_timeslots - unavailable

    T_avail[student] = available

T_unavail_l = pd.read_csv('teacher_unavailable.csv', sep=';').set_index('teachers').groupby('teachers')['timeslots'].apply(set).to_dict()
c = pd.read_csv('teacher_course_competences.csv', sep=';').set_index(['teachers', 'course']).to_dict()['competence']
a = pd.read_csv('student_one_on_one.csv', sep=';').set_index('students').to_dict()['one-on-one']
x_prev = pd.read_csv('previous_week.csv', sep=';').set_index(['students', 'teachers', 'timeslots', 'courses']).to_dict()['assigned']# Model


for s in students:
    for l in teachers:
        for t in timeslots:
            for v in courses:
                if (s, l, t, v) not in x_prev: 
                    x_prev[(s, l, t, v)] = 0  
        
# expand V[s] with imaginary students with no courses
for s in students:
    if s not in V:
        V[s] = []

  


In [2]:
#Phase 1 : Feasibility phase
def calculate_total_objective(x, students, teachers, timeslots, courses, V, R, H, m, M, T_avail_s, T_unavail_l, a):

    y = {}
    y_slv = {}
    # Loop over all students, teachers, timeslots, and courses to compute y_prev
    for s in students:
        for l in teachers:
            for t in timeslots:
                 #Sum over all courses v that student s is assigned to at timeslot t with teacher l
                y[(s, l, t)] = sum(x[s,l,t,v] for v in V[s])
    
    #compute y_slv
    for s in students:
        for l in teachers:      
            for v in V[s]:
                y_slv[s, l, v] = int(any(x[s, l, t, v] == 1 for t in timeslots))
                
    d_st_hc1 = {}
    d_s_hc2 = {}
    d_sv_hc5 = {}
    d_sv_hc6 = {}
    d_sltv_hc7 = {}
    d_slt_hc8 = {}
    d_sltv_hc10 = {}
    d_sltv_hc11 = {}
    d_sv_hc12 = {}


    
    #Assignment d_st_hc1:
    for s in students:
        for t in timeslots:
            total_classes = sum(x[s, l, t, v] for v in V[s] for l in teachers)
            d_st_hc1[s, t] = max(0, total_classes - 1)
        
    #Assignment d_s_hc2:
    for s in students:
        total_hours = sum(x[s, l, t, v] for v in V[s] for l in teachers for t in timeslots)
        d_s_hc2[s] = abs(total_hours - H[s])
    
    #Assignment d_sv_hc5:
    for s in students:
        for v in V[s]:
            d_sv_hc5[s, v] = abs(1 - sum(y_slv[s, l, v] for l in teachers))


    #Assignment d_sv_hc6:
    for s in students:
        for v in V[s]:
            total = sum(x[s, l, t, v] for l in teachers for t in timeslots)
            min_violation = max(0, m[s, v] - total)
            max_violation = max(0, total - M[s, v])
            d_sv_hc6[s, v] = min_violation + max_violation


    #Assignment d_sltv_hc7:
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    d_sltv_hc7[s, l, t, v] = x[s, l, t, v] * (1 - R[s][l])

    # Assignment d_slt_hc8:  
    for s in students:
        for l in teachers:
            for t in timeslots:
                sum_others = sum(x[s_prime, l, t, v_prime] for s_prime in students if s_prime != s for v_prime in V[s_prime])
                rhs = (1 - a[s]) + (1 - y[s, l, t]) * 2
                d_slt_hc8[s, l, t] = max(0, sum_others - rhs)

    #Assignment d_sltv_hc10:
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    if t not in T_avail_s[s]:
                        d_sltv_hc10[s,l,t,v] = x[s,l,t,v]
                    else:
                        d_sltv_hc10[s,l,t,v] = 0

    #Assignment d_sltv_hc11:                    
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    if t in T_unavail_l[l]:
                        d_sltv_hc11[s,l,t,v] = x[s,l,t,v]
                    else:
                        d_sltv_hc11[s,l,t,v] = 0

    #Assignment d_sv_hc12
    for s in students:
        for v in courses:
            if v not in V[s]:
                d_sv_hc12[s,v] = sum(x[s,l,t,v] for l in teachers for t in timeslots)

    total_objective = (
        sum(d_st_hc1.values()) +
        sum(d_s_hc2.values()) +
        sum(d_sv_hc5.values()) +
        sum(d_sv_hc6.values()) +
        sum(d_sltv_hc7.values()) +
        sum(d_slt_hc8.values()) +
        sum(d_sltv_hc10.values()) +
        sum(d_sltv_hc11.values()) +
        sum(d_sv_hc12.values())
    )
    
    return total_objective





In [3]:
def generate_move_set(x, students, teachers, timeslots, courses, V, R, c, T_avail_s, T_unavail_l, H):
    move_set = {
        "insert": set(),
        "delete": set(),
        "reassign_student": set(),
        "reassign_teacher": set(),
        "reassign_timeslot": set(),
        "reassign_subject": set(),
        "swap_students": set(),
        "swap_teachers": set(),
        "swap_timeslots": set(),
        "swap_subjects": set(),
        "swap_teacher_for_course": set()
    }

    for (s, l, t, v), val in x.items():
        if val == 1:
            move_set["delete"].add(("delete", s, l, t, v))

            for s2 in students:
                if s2 != s and H[s2] != 0 and v in V[s2] and R[s2][l] and t in T_avail_s[s2]:
                    move_set["reassign_student"].add(("reassign_student", s, s2, l, t, v))

            for l2 in teachers:
                if l2 != l and R[s][l2] and t not in T_unavail_l[l2]:
                    move_set["reassign_teacher"].add(("reassign_teacher", s, l, l2, t, v))

            for t2 in timeslots:
                if t2 != t and t2 in T_avail_s[s] and t2 not in T_unavail_l[l]:
                    move_set["reassign_timeslot"].add(("reassign_timeslot", s, l, t, t2, v))

            for v2 in V[s]:
                if v2 != v:
                    move_set["reassign_subject"].add(("reassign_subject", s, l, t, v, v2))

    active_assignments = [(s, l, t, v) for (s, l, t, v), val in x.items() if val == 1]

    
    for i in range(len(active_assignments)):
        s1, l1, t1, v1 = active_assignments[i]
        for j in range(i + 1, len(active_assignments)):
            s2, l2, t2, v2 = active_assignments[j]

            # Swap students
            if (s1 != s2 and (l1 != l2 or t1 != t2 or v1 != v2) and v1 in V[s2] and v2 in V[s1] 
                and t1 in T_avail_s[s2] and t2 in T_avail_s[s1] and R[s1][l2] == 1 and R[s2][l1] == 1):
                move_set["swap_students"].add(("swap_students", s1, l1, t1, v1, s2, l2, t2, v2))
    
            # Swap teachers
            if (l1 != l2 and (s1 != s2 or t1 != t2 or v1 != v2) and t1 not in T_unavail_l[l2] 
                and t2 not in T_unavail_l[l1] and R[s1][l2] == 1 and R[s2][l1] == 1):
                move_set["swap_teachers"].add(("swap_teachers", s1, l1, t1, v1, s2, l2, t2, v2))
    
            # Swap timeslots
            if (t1 != t2 and (s1 != s2 or l1 != l2 or v1 != v2) and t2 in T_avail_s[s1] and t1 in T_avail_s[s2]
               and t2 not in T_unavail_l[l1] and t1 not in T_unavail_l[l2]):
                move_set["swap_timeslots"].add(("swap_timeslots", s1, l1, t1, v1, s2, l2, t2, v2))
    
            # Swap subjects
            if (v1 != v2 and (s1 != s2 or t1 != t2 or l1 != l2) and v2 in V[s1] and v1 in V[s2]):
                move_set["swap_subjects"].add(("swap_subjects", s1, l1, t1, v1, s2, l2, t2, v2))

    for s in students:
        for v in V[s]:
            for l1 in teachers:
                # Verzamel alle t waar (s, l1, t, v) actief is
                assigned_t = [t for t in timeslots if x.get((s, l1, t, v), 0) == 1]
                if not assigned_t:
                    continue
                for l2 in teachers:
                    if l1 == l2:
                        continue
                    if R[s][l2] == 1:
                        # Check of l2 beschikbaar is op ALLE t
                        if all(t not in T_unavail_l[l2] for t in assigned_t):
                            move_set["swap_teacher_for_course"].add((
                                "swap_teacher_for_course", s, l1, l2, v
                            ))
  

    for s in students:
        if H[s] == 0:
            continue
        for v in V[s]:
            for l in teachers:
                if R[s][l] == 0:
                    continue
                for t in T_avail_s[s]:
                    if t in T_unavail_l[l]:
                        continue
                    if x.get((s, l, t, v), 0) == 0:
                        move_set["insert"].add(("insert", s, l, t, v))

    return move_set


In [4]:
def update_move_set(move_set, move, x, students, teachers, timeslots, courses, V, R, c, T_avail_s, T_unavail_l, H):
    kind = move[0]

    def add_assignment(s, l, t, v):
        move_set["insert"].discard(("insert", s, l, t, v))
        move_set["delete"].add(("delete", s, l, t, v))
        add_all_related_moves(s, l, t, v)

    def remove_assignment(s, l, t, v):
        move_set["delete"].discard(("delete", s, l, t, v))
        move_set["insert"].add(("insert", s, l, t, v))
        remove_all_related_moves(s, l, t, v)

    def add_all_related_moves(s, l, t, v):
        for s2 in students:
            if s2 != s and H[s2] != 0 and v in V[s2] and R[s2][l] and t in T_avail_s[s2]:
                move_set["reassign_student"].add(("reassign_student", s, s2, l, t, v))
        for l2 in teachers:
            if l2 != l and R[s][l2] and t not in T_unavail_l[l2]:
                move_set["reassign_teacher"].add(("reassign_teacher", s, l, l2, t, v))
        for t2 in timeslots:
            if t2 != t and t2 in T_avail_s[s] and t2 not in T_unavail_l[l]:
                move_set["reassign_timeslot"].add(("reassign_timeslot", s, l, t, t2, v))
        for v2 in V[s]:
            if v2 != v:
                move_set["reassign_subject"].add(("reassign_subject", s, l, t, v, v2))

        active = [(s2, l2, t2, v2) for (s2, l2, t2, v2), val in x.items() if val == 1 and (s2, l2, t2, v2) != (s, l, t, v)]
        for s2, l2, t2, v2 in active:
            if s != s2 and (l != l2 or t != t2 or v != v2) and v in V[s2] and v2 in V[s] and \
               t in T_avail_s[s2] and t2 in T_avail_s[s] and R[s][l2] and R[s2][l]:
                move_set["swap_students"].add(("swap_students", s, l, t, v, s2, l2, t2, v2))
            if l != l2 and (s != s2 or t != t2 or v != v2) and \
               t not in T_unavail_l[l2] and t2 not in T_unavail_l[l] and R[s][l2] and R[s2][l]:
                move_set["swap_teachers"].add(("swap_teachers", s, l, t, v, s2, l2, t2, v2))
            if t != t2 and (s != s2 or l != l2 or v != v2) and \
               t2 in T_avail_s[s] and t in T_avail_s[s2] and t2 not in T_unavail_l[l] and t not in T_unavail_l[l2]:
                move_set["swap_timeslots"].add(("swap_timeslots", s, l, t, v, s2, l2, t2, v2))
            if v != v2 and (s != s2 or t != t2 or l != l2) and v2 in V[s] and v in V[s2]:
                move_set["swap_subjects"].add(("swap_subjects", s, l, t, v, s2, l2, t2, v2))

        for l2 in teachers:
            if l2 != l and R[s][l2] and t not in T_unavail_l[l2]:
                move_set["swap_teacher_for_course"].add(("swap_teacher_for_course", s, l, l2, v))

    def remove_all_related_moves(s, l, t, v):
        
        for m in list(move_set["reassign_student"]):
            _, s1, s2, l_, t_, v_ = m
            if s1 == s and l_ == l and t_ == t and v_ == v:
                move_set["reassign_student"].discard(m)

        for m in list(move_set["reassign_teacher"]):
            _, s_, l1, l2, t_, v_ = m
            if s_ == s and l1 == l and t_ == t and v_ == v:
                move_set["reassign_teacher"].discard(m)

        for m in list(move_set["reassign_timeslot"]):
            _, s_, l_, t1, t2, v_ = m
            if s_ == s and l_ == l and t1 == t and v_ == v:
                move_set["reassign_timeslot"].discard(m)

        for m in list(move_set["reassign_subject"]):
            _, s_, l_, t_, v1, v2 = m
            if s_ == s and l_ == l and t_ == t and v1 == v:
                move_set["reassign_subject"].discard(m)

        for move_type in ["swap_students", "swap_teachers", "swap_timeslots", "swap_subjects"]:
            for m in list(move_set[move_type]):
                _, s1, l1, t1, v1, s2, l2, t2, v2 = m
                if (s1 == s and l1 == l and t1 == t and v1 == v) or (s2 == s and l2 == l and t2 == t and v2 == v):
                    move_set[move_type].discard(m)


        for m in list(move_set["swap_teacher_for_course"]):
            _, s_, l1, l2, v_ = m
            if s_ == s and l1 == l and v_ == v:
                move_set["swap_teacher_for_course"].discard(m)

    # === Main logic for all move types ===

    if kind == "insert":
        _, s, l, t, v = move
        add_assignment(s, l, t, v)

    elif kind == "delete":
        _, s, l, t, v = move
        remove_assignment(s, l, t, v)

    elif kind == "reassign_student":
        _, s1, s2, l, t, v = move
        remove_assignment(s1, l, t, v)
        add_assignment(s2, l, t, v)

    elif kind == "reassign_teacher":
        _, s, l1, l2, t, v = move
        remove_assignment(s, l1, t, v)
        add_assignment(s, l2, t, v)

    elif kind == "reassign_timeslot":
        _, s, l, t1, t2, v = move
        remove_assignment(s, l, t1, v)
        add_assignment(s, l, t2, v)

    elif kind == "reassign_subject":
        _, s, l, t, v1, v2 = move
        remove_assignment(s, l, t, v1)
        add_assignment(s, l, t, v2)

    elif kind in {"swap_students", "swap_teachers", "swap_timeslots", "swap_subjects"}:
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        remove_assignment(s1, l1, t1, v1)
        remove_assignment(s2, l2, t2, v2)

        if kind == "swap_students":
            add_assignment(s2, l1, t1, v1)
            add_assignment(s1, l2, t2, v2)
        elif kind == "swap_teachers":
            add_assignment(s1, l2, t1, v1)
            add_assignment(s2, l1, t2, v2)
        elif kind == "swap_timeslots":
            add_assignment(s1, l1, t2, v1)
            add_assignment(s2, l2, t1, v2)
        elif kind == "swap_subjects":
            add_assignment(s1, l1, t1, v2)
            add_assignment(s2, l2, t2, v1)

    elif kind == "swap_teacher_for_course":
        _, s, l1, l2, v = move
        for t in timeslots:
            if x.get((s, l1, t, v), 0) == 1:
                remove_assignment(s, l1, t, v)
                add_assignment(s, l2, t, v)

    return move_set


In [5]:
import copy

def apply_move(x, move):
    x_new = copy.deepcopy(x)
    move_type = move[0]

    if move_type == "insert":
        _, s, l, t, v = move
        x_new[s, l, t, v] = 1

    elif move_type == "delete":
        _, s, l, t, v = move
        x_new[s, l, t, v] = 0
        
    elif move_type == "reassign_student":
        _, s_old, s_new, l, t, v = move
        x_new[s_old, l, t, v] = 0
        x_new[s_new, l, t, v] = 1

    elif move_type == "reassign_teacher":
        _, s, l_old, l_new, t, v = move
        x_new[s, l_old, t, v] = 0
        x_new[s, l_new, t, v] = 1

    elif move_type == "reassign_timeslot":
        _, s, l, t_old, t_new, v = move
        x_new[s, l, t_old, v] = 0
        x_new[s, l, t_new, v] = 1

    elif move_type == "reassign_subject":
        _, s, l, t, v_old, v_new = move
        x_new[s, l, t, v_old] = 0
        x_new[s, l, t, v_new] = 1

    elif move_type == "swap_students":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l2, t2, v2], x_new[s2, l1, t1, v1] = 1, 1

    elif move_type == "swap_teachers":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l2, t2, v2], x_new[s2, l1, t1, v1] = 1, 1

    elif move_type == "swap_subjects":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l1, t1, v2], x_new[s2, l2, t2, v1] = 1, 1

    elif move_type == "swap_timeslots":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l1, t2, v1], x_new[s2, l2, t1, v2] = 1, 1

    elif move_type == "swap_teacher_for_course":
        _, s, l1, l2, v = move
        for t in timeslots:
            if x[s, l1, t, v] == 1:
                x_new[s, l1, t, v] = 0
                x_new[s, l2, t, v] = 1

    return x_new


In [6]:
def local_search(x_start, students, teachers, timeslots, courses, V, R, H, m, M, T_avail_s, T_unavail_l, a):
    import copy
    import random

    
    x_current = copy.deepcopy(x_start)
    obj = calculate_total_objective(x_current, students, teachers, timeslots, courses, V, R, H, m, M, T_avail_s, T_unavail_l, a)
    print(f"Initial objective: {obj}")

    move_set = generate_move_set(x_current, students, teachers, timeslots, courses, V, R, c, T_avail_s, T_unavail_l, H)

    while obj > 0:
        found_improvement = False

        MAX_MOVES = 10000

        all_moves = []
        
        for move_type in move_set:
            all_moves.extend([(move_type, move) for move in move_set[move_type]])

        sampled_moves = random.sample(all_moves, min(MAX_MOVES, len(all_moves)))

        for move_type, move in sampled_moves:
 
            x_candidate = apply_move(x_current, move)
            obj_candidate = calculate_total_objective(x_candidate, students, teachers, timeslots, courses, V, R, H, m, M, T_avail_s, T_unavail_l, a)

            if obj_candidate < obj:
                x_current = x_candidate
                obj = obj_candidate
                update_move_set(move_set, move, x_current, students, teachers, timeslots, courses, V, R, c, T_avail_s, T_unavail_l, H)
                print(f"Applied {move_type} move: {move} | New objective: {obj}")
                found_improvement = True
                break


        if not found_improvement:
            print("No improving move found. Local minimum reached.")
            break

    return x_current, obj


In [7]:
x_final, final_obj = local_search(
    x_prev, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a
)

print("Feasibility phase search complete. Final objective:", final_obj)

Initial objective: 134.0
Applied insert move: ('insert', 's58', 'l5', 10, 'Nederlands 2de graad') | New objective: 131.0
Applied reassign_timeslot move: ('reassign_timeslot', 's26', 'l7', 12.0, 22, 'Varia') | New objective: 130.0
Applied insert move: ('insert', 's43', 'l8', 17, 'PAV Algemeen 3de graad') | New objective: 129.0
Applied reassign_teacher move: ('reassign_teacher', 's4', 'l2', 'l7', 9.0, 'Spaans') | New objective: 128.0
Applied reassign_timeslot move: ('reassign_timeslot', 's49', 'l1', 24.0, 5, 'Wiskunde lager') | New objective: 127.0
Applied insert move: ('insert', 's34', 'l3', 16, 'Gezond en Welzijn 2de graad') | New objective: 125.0
Applied insert move: ('insert', 's58', 'l4', 1, 'Nederlands 2de graad') | New objective: 124.0
Applied insert move: ('insert', 's41', 'l11', 23, 'Praktijk Atelier') | New objective: 123.0
Applied insert move: ('insert', 's44', 'l4', 2, 'Plantenkennis') | New objective: 122.0
Applied insert move: ('insert', 's37', 'l1', 13, 'Zelfstudie') | New

KeyboardInterrupt: 